In [1]:
import json
import jieba
import re
import numpy as np
import pandas as pd
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

In [2]:
# 这是用来生成lsa向量的训练集，必须用这个数据集来拼接lsa向量
train_data_lsa_cvjd_54339 = pd.read_pickle('../generalization_data/train_data_lsa_cvjd_54339.pkl')
train_data_lsa_cvjd_54339

,cv_id,jd_id,jd_code,title,deadline,category_id,category_name,company_id,company_name,recruit_number,...,workYear,avatar,eduTracks,jobTracks,projectTracks,ds.1,da_label,rank_label,resume_type,order_month
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,JD20210910115615503390,市场营销负责人,2021-11-30 23:59:59.0,140101,市场营销,2c9207157bcbb76b017bcdd5f254038a,深圳市快易典教育科技有限公司,1,...,13.0,NaN,"['{""degree"": ""本科"", ""endDate"": ""2008-07"", ""id"":...","['{""companyName"": ""深圳市亿科数字科技有限公司"", ""descriptio...","['{""companyName"": ""*深圳市蜂联科技有限公司（奇虎360）"", ""desc...",20220831,3,3,NaN,2021-09
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,JD20211126124504034196,海外区域运营经理/高级经理,2022-02-28 23:59:59.0,130000,运营,2c9207157bf94d99017c1714f34e1715,广州市百果园网络科技有限公司,1,...,3.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2019-06"", ""id"":...","['{""companyName"": ""北京星制科技有限公司（全民快乐）"", ""descrip...","['{""description"": ""主播表现，直播平均 2 场，平台新用户 12926，主...",20220831,-1,2,4.0,2021-12
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,JD20211130113932574529,高级项目经理,NaN,310000,高级管理,2c9207157bf94d99017c2f902e952262,深圳纳欣科技有限公司,1,...,5.0,https://image0.lietou-static.com/big/5f8fa3bde...,"['{""degree"": ""硕士"", ""endDate"": ""2016-07"", ""id"":...","['{""companyName"": ""酷狗音乐"", ""companyTags"": ""互联网+...","['{""companyName"": ""酷狗音乐"", ""description"": ""酷狗20...",20220831,1,1,4.0,2021-12
5,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,JD20211223141525111469,阿里云智能-金融行业解决方案架构师（公有云）,NaN,100405,IT技术支持,2c9207157ccbcd86017d098f2b6454be,阿里云计算有限公司,1,...,11.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":...","['{""companyName"": ""华为技术有限公司"", ""department"": ""年...","['{""description"": ""项目职务: 项目经理\\n项目描述: 引导公司 IT ...",20220831,2,2,4.0,2021-12
6,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,JD20211216113026318551,金融行业解决方案架构师,NaN,101201,售前技术支持,2c9207157ccbcd86017d098f2b6454be,阿里云计算有限公司,2,...,11.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":...","['{""companyName"": ""华为技术有限公司"", ""department"": ""年...","['{""description"": ""项目职务: 项目经理\\n项目描述: 引导公司 IT ...",20220831,1,1,4.0,2021-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82315,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,JD20220725145055775181,C++开发工程师-智能创作,NaN,100102,C++,2c9207157d95bfde017da1e665a9550d,北京字节跳动科技有限公司,1,...,9.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""2022-01"", ""...","['{""endDate"": ""2022-07"", ""companyName"": ""北京开课吧...","['{""description"": ""项目说明: 短信平台是开课吧公司级短信需求处理的基础服...",20220831,2,2,4.0,2022-07
82316,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,JD20220719100753718175,结构总监/经理,NaN,300800,新能源,2c92071581ebf3390182010c347d60ea,深圳市首航新能源股份有限公司,1,...,16.0,NaN,"['{""endDate"": ""2007-07"", ""unified"": true, ""deg...","['{""endDate"": ""9999-01"", ""companyName"": ""深圳市正浩...","['{""endDate"": ""2013-03"", ""companyName"": ""深圳市方格...",20220831,1,1,4.0,2022-07
82317,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,JD20220721172428481465,HRBP,NaN,150103,HRBP,8a69f68e82198e910182200c3e497c5c,深圳微诚控股集团有限公司,1,...,4.0,NaN,"['{""endDate"": ""2018-07"", ""unified"": true, ""deg...","['{""endDate"": ""2022-03"", ""companyName"": ""奥园集团佛...","['{""endDate"": ""2022-03"", ""description"": ""工作内容:...",20220831,-1,2,4.0,2022-07
82320,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,JD20220725190337904313,产品经理,NaN,110100,产品经理,2c9207158114fb29018132ace5f11fe9,广州视源电子科技股份有限公司,1,...,3.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""2019-06"", ""...","['{""endDate"": ""9999-01"", ""companyName"": ""美的"", ...","['{""name"": null}']",20220831,2,2,4.0,2022-07


In [4]:
train_data_lsa_cvjd_54339[['cv_id', 'jd_id', 'title', 'category_name', 'tags']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54339 entries, 0 to 82322
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   cv_id          54339 non-null  object
 1   jd_id          54339 non-null  object
 2   title          54339 non-null  object
 3   category_name  54339 non-null  object
 4   tags           53867 non-null  object
dtypes: object(5)
memory usage: 2.5+ MB


In [3]:
train_data_lsa_cvjd_54339.reset_index(drop=True)

,cv_id,jd_id,jd_code,title,deadline,category_id,category_name,company_id,company_name,recruit_number,...,workYear,avatar,eduTracks,jobTracks,projectTracks,ds.1,da_label,rank_label,resume_type,order_month
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,JD20210910115615503390,市场营销负责人,2021-11-30 23:59:59.0,140101,市场营销,2c9207157bcbb76b017bcdd5f254038a,深圳市快易典教育科技有限公司,1,...,13.0,NaN,"['{""degree"": ""本科"", ""endDate"": ""2008-07"", ""id"":...","['{""companyName"": ""深圳市亿科数字科技有限公司"", ""descriptio...","['{""companyName"": ""*深圳市蜂联科技有限公司（奇虎360）"", ""desc...",20220831,3,3,NaN,2021-09
1,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,JD20211126124504034196,海外区域运营经理/高级经理,2022-02-28 23:59:59.0,130000,运营,2c9207157bf94d99017c1714f34e1715,广州市百果园网络科技有限公司,1,...,3.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2019-06"", ""id"":...","['{""companyName"": ""北京星制科技有限公司（全民快乐）"", ""descrip...","['{""description"": ""主播表现，直播平均 2 场，平台新用户 12926，主...",20220831,-1,2,4.0,2021-12
2,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,JD20211130113932574529,高级项目经理,NaN,310000,高级管理,2c9207157bf94d99017c2f902e952262,深圳纳欣科技有限公司,1,...,5.0,https://image0.lietou-static.com/big/5f8fa3bde...,"['{""degree"": ""硕士"", ""endDate"": ""2016-07"", ""id"":...","['{""companyName"": ""酷狗音乐"", ""companyTags"": ""互联网+...","['{""companyName"": ""酷狗音乐"", ""description"": ""酷狗20...",20220831,1,1,4.0,2021-12
3,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,JD20211223141525111469,阿里云智能-金融行业解决方案架构师（公有云）,NaN,100405,IT技术支持,2c9207157ccbcd86017d098f2b6454be,阿里云计算有限公司,1,...,11.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":...","['{""companyName"": ""华为技术有限公司"", ""department"": ""年...","['{""description"": ""项目职务: 项目经理\\n项目描述: 引导公司 IT ...",20220831,2,2,4.0,2021-12
4,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,JD20211216113026318551,金融行业解决方案架构师,NaN,101201,售前技术支持,2c9207157ccbcd86017d098f2b6454be,阿里云计算有限公司,2,...,11.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":...","['{""companyName"": ""华为技术有限公司"", ""department"": ""年...","['{""description"": ""项目职务: 项目经理\\n项目描述: 引导公司 IT ...",20220831,1,1,4.0,2021-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54334,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,JD20220725145055775181,C++开发工程师-智能创作,NaN,100102,C++,2c9207157d95bfde017da1e665a9550d,北京字节跳动科技有限公司,1,...,9.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""2022-01"", ""...","['{""endDate"": ""2022-07"", ""companyName"": ""北京开课吧...","['{""description"": ""项目说明: 短信平台是开课吧公司级短信需求处理的基础服...",20220831,2,2,4.0,2022-07
54335,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,JD20220719100753718175,结构总监/经理,NaN,300800,新能源,2c92071581ebf3390182010c347d60ea,深圳市首航新能源股份有限公司,1,...,16.0,NaN,"['{""endDate"": ""2007-07"", ""unified"": true, ""deg...","['{""endDate"": ""9999-01"", ""companyName"": ""深圳市正浩...","['{""endDate"": ""2013-03"", ""companyName"": ""深圳市方格...",20220831,1,1,4.0,2022-07
54336,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,JD20220721172428481465,HRBP,NaN,150103,HRBP,8a69f68e82198e910182200c3e497c5c,深圳微诚控股集团有限公司,1,...,4.0,NaN,"['{""endDate"": ""2018-07"", ""unified"": true, ""deg...","['{""endDate"": ""2022-03"", ""companyName"": ""奥园集团佛...","['{""endDate"": ""2022-03"", ""description"": ""工作内容:...",20220831,-1,2,4.0,2022-07
54337,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,JD20220725190337904313,产品经理,NaN,110100,产品经理,2c9207158114fb29018132ace5f11fe9,广州视源电子科技股份有限公司,1,...,3.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""2019-06"", ""...","['{""endDate"": ""9999-01"", ""companyName"": ""美的"", ...","['{""name"": null}']",20220831,2,2,4.0,2022-07


In [4]:
pd.read_pickle('../generalization_data/train_description_tfidf_lsa.pkl')

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
0,0.245978,0.219388,-0.001255,0.060505,-0.019605,-0.018920,-0.069819,0.049438,-0.047138,0.038305,...,-0.008249,0.001610,-0.021326,0.023228,-0.005978,0.010373,-0.008609,-0.009598,-0.018672,-0.004692
1,0.263932,0.104354,-0.086183,-0.067204,0.090540,0.068917,-0.058438,0.039966,-0.050826,0.062841,...,-0.038992,0.014570,-0.031770,-0.002013,0.000193,0.003466,-0.000330,-0.030817,0.007213,-0.004130
2,0.276421,-0.027352,0.082342,0.081767,-0.026445,0.040150,-0.166881,-0.146737,-0.006918,0.022834,...,-0.011792,0.019838,0.058874,0.046240,0.001635,0.034112,-0.049644,-0.012443,-0.045069,-0.055091
3,0.203287,-0.005198,-0.064189,0.127611,0.017325,-0.101736,0.151930,-0.042431,-0.012835,0.016266,...,0.027772,-0.062224,-0.041545,-0.031999,-0.027054,0.015455,0.006008,0.053297,-0.020676,-0.038348
4,0.204987,-0.006300,-0.064196,0.126072,0.016448,-0.101227,0.151094,-0.042071,-0.013972,0.017931,...,0.025657,-0.063846,-0.038687,-0.031527,-0.029348,0.016352,0.003930,0.049575,-0.028572,-0.034439
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54334,0.262550,-0.085161,0.077126,0.071275,-0.005347,-0.078708,0.019406,-0.024365,-0.076055,0.089592,...,-0.030109,-0.018343,0.008360,0.014661,-0.005713,-0.039473,-0.037732,-0.022184,-0.065903,0.058242
54335,0.134888,-0.043534,0.027937,0.016888,-0.039027,0.016181,-0.029961,-0.051793,0.005803,0.001074,...,0.006023,0.019793,0.022524,0.013595,0.013955,0.036088,0.008624,0.018415,-0.000551,0.002733
54336,0.192223,0.250674,0.053165,-0.120097,-0.116748,-0.052941,0.113171,0.097596,-0.007032,0.012874,...,0.062382,0.007490,-0.021115,0.015207,0.026372,-0.021530,0.014231,0.024896,-0.033106,-0.030288
54337,0.255035,-0.094685,-0.170120,-0.007026,-0.024592,-0.049753,0.043363,-0.068608,-0.007928,-0.002506,...,0.009836,0.030863,-0.019922,-0.030850,0.015589,-0.032353,0.030428,0.053683,0.000512,-0.012981


# tfidf_lsa

In [5]:
# 读取生成的5份tfidf_pca文件，并拼接
# 注意，不是join的时候，concat是通过index拼接的。所以需要将索引重置。
sample = train_data_lsa_cvjd_54339.reset_index(drop=True)

# 需要将这个向量变成一个列表
def tfidf_lsa_merge_fun(series):
    return list(series)

tfidf_lsa_file = ['train_description_tfidf_lsa.pkl', 'train_position_tfidf_lsa.pkl', 'train_requirement_tfidf_lsa.pkl', 'train_skills_tfidf_lsa.pkl', 'train_title_category_tags_tfidf_lsa.pkl']
for pkl_file in tfidf_lsa_file:
    tfidf_lsa = pd.read_pickle('../generalization_data/'+pkl_file)
    print(pkl_file)
    #print(tfidf_pca)
    col_name = pkl_file.strip('.pkl') # 把skills的s去掉了
    tfidf_lsa[col_name] = tfidf_lsa.apply(tfidf_lsa_merge_fun, axis=1)
    sample = pd.concat([sample, tfidf_lsa[[col_name]]], axis=1)

tfidf_lsa_col_names = [pkl_file.strip('.pkl') for pkl_file in tfidf_lsa_file]
sample[['cv_id', 'jd_id']+tfidf_lsa_col_names]

train_description_tfidf_lsa.pkl
train_position_tfidf_lsa.pkl
train_requirement_tfidf_lsa.pkl
train_skills_tfidf_lsa.pkl
train_title_category_tags_tfidf_lsa.pkl


,cv_id,jd_id,train_description_tfidf_lsa,train_position_tfidf_lsa,train_requirement_tfidf_lsa,train_skills_tfidf_lsa,train_title_category_tags_tfidf_lsa
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[0.2459783180342179, 0.21938831787537966, -0.0...","[0.007319529543557632, 0.0003354154328290771, ...","[0.033013137623400135, -0.035688917703485826, ...","[0.53765937650696, -0.14639534434021456, -0.07...","[0.139618999709323, -0.030086416810889256, -0...."
1,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[0.26393201542159855, 0.10435436209119428, -0....","[0.0995745047074556, -0.02088326554704094, -0....","[0.37018351050902737, -0.20255122674942938, -0...","[0.236061264433608, -0.12857925603594755, 0.13...","[0.5419245911087135, -0.18238753835825924, -0...."
2,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.27642059835889154, -0.02735173829637042, 0....","[0.043222795997538296, 0.01108309987508924, 0....","[0.28941341021190825, -0.0474103810249287, -0....","[0.00406004143441403, 0.005098830611803655, 0....","[0.09166404085822458, 0.013460082203324779, -0..."
3,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,"[0.20328670858820733, -0.005197850015064169, -...","[0.15026256194834306, -0.004905085247130754, -...","[0.18618173823787507, 0.031168513777247584, 0....","[0.1555934824946681, 0.15537445471422626, -0.0...","[0.08785093521473429, 0.04596513112440933, -0...."
4,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,"[0.20498743446953704, -0.006300276619096516, -...","[0.15026256194834306, -0.004905085247130754, -...","[0.19163011557219942, 0.025951225287628978, 0....","[0.1555934824946681, 0.15537445471422626, -0.0...","[0.10718268355356483, 0.05565724707601688, -0...."
...,...,...,...,...,...,...,...
54334,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,"[0.2625502866681783, -0.08516054875935031, 0.0...","[0.23530919072669182, 0.7944767263661332, 0.01...","[0.30282726216339745, 0.16097547637426476, 0.0...","[0.27210694684652126, 0.42441791865396394, -0....","[0.08847069778780742, 0.22734187469570663, -0...."
54335,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,"[0.13488778904906526, -0.043533761211661984, 0...","[0.6087412984522569, -0.1473860953415076, -0.0...","[0.15424621070300917, -0.0012759763333448448, ...","[0.2628487548869251, 0.0075222509233238725, -0...","[0.19044431984360052, -0.019572235686217573, 0..."
54336,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,"[0.192223443587156, 0.25067413083354817, 0.053...","[0.024225991980613454, -0.0015968824512811737,...","[0.23573037593299082, -0.13812537376984757, 0....","[0.3526478953565766, -0.20899689717605643, 0.1...","[0.1171332341680559, -0.006169459729397869, -0..."
54337,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,"[0.2550351130980516, -0.09468529288217858, -0....","[0.8678408186646723, -0.22318129673531545, -0....","[0.19019357171237353, -0.12296867979940661, -0...","[0.27948075994260324, -0.04971016234222427, 0....","[0.3466942038817421, -0.07639258814853622, 0.3..."


In [6]:
sample[['cv_id', 'jd_id']+tfidf_lsa_col_names].to_pickle('../generalization_data/train_data_lsa_cvjd_merge_54339.pkl')

In [7]:
cvjd_lsa = sample[['cv_id', 'jd_id']+tfidf_lsa_col_names]
cvjd_lsa

,cv_id,jd_id,train_description_tfidf_lsa,train_position_tfidf_lsa,train_requirement_tfidf_lsa,train_skills_tfidf_lsa,train_title_category_tags_tfidf_lsa
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[0.2459783180342179, 0.21938831787537966, -0.0...","[0.007319529543557632, 0.0003354154328290771, ...","[0.033013137623400135, -0.035688917703485826, ...","[0.53765937650696, -0.14639534434021456, -0.07...","[0.139618999709323, -0.030086416810889256, -0...."
1,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[0.26393201542159855, 0.10435436209119428, -0....","[0.0995745047074556, -0.02088326554704094, -0....","[0.37018351050902737, -0.20255122674942938, -0...","[0.236061264433608, -0.12857925603594755, 0.13...","[0.5419245911087135, -0.18238753835825924, -0...."
2,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.27642059835889154, -0.02735173829637042, 0....","[0.043222795997538296, 0.01108309987508924, 0....","[0.28941341021190825, -0.0474103810249287, -0....","[0.00406004143441403, 0.005098830611803655, 0....","[0.09166404085822458, 0.013460082203324779, -0..."
3,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,"[0.20328670858820733, -0.005197850015064169, -...","[0.15026256194834306, -0.004905085247130754, -...","[0.18618173823787507, 0.031168513777247584, 0....","[0.1555934824946681, 0.15537445471422626, -0.0...","[0.08785093521473429, 0.04596513112440933, -0...."
4,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,"[0.20498743446953704, -0.006300276619096516, -...","[0.15026256194834306, -0.004905085247130754, -...","[0.19163011557219942, 0.025951225287628978, 0....","[0.1555934824946681, 0.15537445471422626, -0.0...","[0.10718268355356483, 0.05565724707601688, -0...."
...,...,...,...,...,...,...,...
54334,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,"[0.2625502866681783, -0.08516054875935031, 0.0...","[0.23530919072669182, 0.7944767263661332, 0.01...","[0.30282726216339745, 0.16097547637426476, 0.0...","[0.27210694684652126, 0.42441791865396394, -0....","[0.08847069778780742, 0.22734187469570663, -0...."
54335,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,"[0.13488778904906526, -0.043533761211661984, 0...","[0.6087412984522569, -0.1473860953415076, -0.0...","[0.15424621070300917, -0.0012759763333448448, ...","[0.2628487548869251, 0.0075222509233238725, -0...","[0.19044431984360052, -0.019572235686217573, 0..."
54336,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,"[0.192223443587156, 0.25067413083354817, 0.053...","[0.024225991980613454, -0.0015968824512811737,...","[0.23573037593299082, -0.13812537376984757, 0....","[0.3526478953565766, -0.20899689717605643, 0.1...","[0.1171332341680559, -0.006169459729397869, -0..."
54337,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,"[0.2550351130980516, -0.09468529288217858, -0....","[0.8678408186646723, -0.22318129673531545, -0....","[0.19019357171237353, -0.12296867979940661, -0...","[0.27948075994260324, -0.04971016234222427, 0....","[0.3466942038817421, -0.07639258814853622, 0.3..."


# work_duration equal_job salary

In [8]:
work_duration_mean_df = pd.read_pickle('../generalization_data/train_work_duration_mean.pkl')
equal_job_df = pd.read_pickle('../generalization_data/train_equal_job.pkl')
salary_df = pd.read_pickle('../generalization_data/train_salary.pkl')

In [9]:
cvjd_lsa_work_duration_equal_job_salary = cvjd_lsa.\
    join(work_duration_mean_df.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner').\
    join(equal_job_df.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner').\
    join(salary_df.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')

cvjd_lsa_work_duration_equal_job_salary

,cv_id,jd_id,train_description_tfidf_lsa,train_position_tfidf_lsa,train_requirement_tfidf_lsa,train_skills_tfidf_lsa,train_title_category_tags_tfidf_lsa,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[0.2459783180342179, 0.21938831787537966, -0.0...","[0.007319529543557632, 0.0003354154328290771, ...","[0.033013137623400135, -0.035688917703485826, ...","[0.53765937650696, -0.14639534434021456, -0.07...","[0.139618999709323, -0.030086416810889256, -0....",1209.5,3,-1,-1
1,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[0.26393201542159855, 0.10435436209119428, -0....","[0.0995745047074556, -0.02088326554704094, -0....","[0.37018351050902737, -0.20255122674942938, -0...","[0.236061264433608, -0.12857925603594755, 0.13...","[0.5419245911087135, -0.18238753835825924, -0....",76.5,3,-1,-1
2,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.27642059835889154, -0.02735173829637042, 0....","[0.043222795997538296, 0.01108309987508924, 0....","[0.28941341021190825, -0.0474103810249287, -0....","[0.00406004143441403, 0.005098830611803655, 0....","[0.09166404085822458, 0.013460082203324779, -0...",1034.0,1,450,-1
3,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,"[0.20328670858820733, -0.005197850015064169, -...","[0.15026256194834306, -0.004905085247130754, -...","[0.18618173823787507, 0.031168513777247584, 0....","[0.1555934824946681, 0.15537445471422626, -0.0...","[0.08785093521473429, 0.04596513112440933, -0....",NaN,2,-1,-1
4,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,"[0.20498743446953704, -0.006300276619096516, -...","[0.15026256194834306, -0.004905085247130754, -...","[0.19163011557219942, 0.025951225287628978, 0....","[0.1555934824946681, 0.15537445471422626, -0.0...","[0.10718268355356483, 0.05565724707601688, -0....",NaN,2,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...
54334,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,"[0.2625502866681783, -0.08516054875935031, 0.0...","[0.23530919072669182, 0.7944767263661332, 0.01...","[0.30282726216339745, 0.16097547637426476, 0.0...","[0.27210694684652126, 0.42441791865396394, -0....","[0.08847069778780742, 0.22734187469570663, -0....",1075.0,5,-1,300
54335,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,"[0.13488778904906526, -0.043533761211661984, 0...","[0.6087412984522569, -0.1473860953415076, -0.0...","[0.15424621070300917, -0.0012759763333448448, ...","[0.2628487548869251, 0.0075222509233238725, -0...","[0.19044431984360052, -0.019572235686217573, 0...",796.5,7,1020,850
54336,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,"[0.192223443587156, 0.25067413083354817, 0.053...","[0.024225991980613454, -0.0015968824512811737,...","[0.23573037593299082, -0.13812537376984757, 0....","[0.3526478953565766, -0.20899689717605643, 0.1...","[0.1171332341680559, -0.006169459729397869, -0...",1019.0,1,600,120
54337,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,"[0.2550351130980516, -0.09468529288217858, -0....","[0.8678408186646723, -0.22318129673531545, -0....","[0.19019357171237353, -0.12296867979940661, -0...","[0.27948075994260324, -0.04971016234222427, 0....","[0.3466942038817421, -0.07639258814853622, 0.3...",NaN,4,280,240


# new feature

In [10]:
# 保存为new feature
cvjd_lsa_work_duration_equal_job_salary.to_pickle('../generalization_data/train_new_feature_54339.pkl')


# raw feature

In [11]:
# 这是用来生成lsa向量的训练集，必须用这个数据集来拼接lsa向量
# 这份文件其实是用raw_cvjd的数据和train id进行join得到的
train_data_lsa_cvjd_54339 = pd.read_pickle('../generalization_data/train_data_lsa_cvjd_54339.pkl')
train_data_lsa_cvjd_54339

,cv_id,jd_id,jd_code,title,deadline,category_id,category_name,company_id,company_name,recruit_number,...,workYear,avatar,eduTracks,jobTracks,projectTracks,ds.1,da_label,rank_label,resume_type,order_month
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,JD20210910115615503390,市场营销负责人,2021-11-30 23:59:59.0,140101,市场营销,2c9207157bcbb76b017bcdd5f254038a,深圳市快易典教育科技有限公司,1,...,13.0,NaN,"['{""degree"": ""本科"", ""endDate"": ""2008-07"", ""id"":...","['{""companyName"": ""深圳市亿科数字科技有限公司"", ""descriptio...","['{""companyName"": ""*深圳市蜂联科技有限公司（奇虎360）"", ""desc...",20220831,3,3,NaN,2021-09
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,JD20211126124504034196,海外区域运营经理/高级经理,2022-02-28 23:59:59.0,130000,运营,2c9207157bf94d99017c1714f34e1715,广州市百果园网络科技有限公司,1,...,3.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2019-06"", ""id"":...","['{""companyName"": ""北京星制科技有限公司（全民快乐）"", ""descrip...","['{""description"": ""主播表现，直播平均 2 场，平台新用户 12926，主...",20220831,-1,2,4.0,2021-12
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,JD20211130113932574529,高级项目经理,NaN,310000,高级管理,2c9207157bf94d99017c2f902e952262,深圳纳欣科技有限公司,1,...,5.0,https://image0.lietou-static.com/big/5f8fa3bde...,"['{""degree"": ""硕士"", ""endDate"": ""2016-07"", ""id"":...","['{""companyName"": ""酷狗音乐"", ""companyTags"": ""互联网+...","['{""companyName"": ""酷狗音乐"", ""description"": ""酷狗20...",20220831,1,1,4.0,2021-12
5,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,JD20211223141525111469,阿里云智能-金融行业解决方案架构师（公有云）,NaN,100405,IT技术支持,2c9207157ccbcd86017d098f2b6454be,阿里云计算有限公司,1,...,11.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":...","['{""companyName"": ""华为技术有限公司"", ""department"": ""年...","['{""description"": ""项目职务: 项目经理\\n项目描述: 引导公司 IT ...",20220831,2,2,4.0,2021-12
6,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,JD20211216113026318551,金融行业解决方案架构师,NaN,101201,售前技术支持,2c9207157ccbcd86017d098f2b6454be,阿里云计算有限公司,2,...,11.0,NaN,"['{""degree"": ""硕士"", ""endDate"": ""2010-07"", ""id"":...","['{""companyName"": ""华为技术有限公司"", ""department"": ""年...","['{""description"": ""项目职务: 项目经理\\n项目描述: 引导公司 IT ...",20220831,1,1,4.0,2021-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82315,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,JD20220725145055775181,C++开发工程师-智能创作,NaN,100102,C++,2c9207157d95bfde017da1e665a9550d,北京字节跳动科技有限公司,1,...,9.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""2022-01"", ""...","['{""endDate"": ""2022-07"", ""companyName"": ""北京开课吧...","['{""description"": ""项目说明: 短信平台是开课吧公司级短信需求处理的基础服...",20220831,2,2,4.0,2022-07
82316,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,JD20220719100753718175,结构总监/经理,NaN,300800,新能源,2c92071581ebf3390182010c347d60ea,深圳市首航新能源股份有限公司,1,...,16.0,NaN,"['{""endDate"": ""2007-07"", ""unified"": true, ""deg...","['{""endDate"": ""9999-01"", ""companyName"": ""深圳市正浩...","['{""endDate"": ""2013-03"", ""companyName"": ""深圳市方格...",20220831,1,1,4.0,2022-07
82317,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,JD20220721172428481465,HRBP,NaN,150103,HRBP,8a69f68e82198e910182200c3e497c5c,深圳微诚控股集团有限公司,1,...,4.0,NaN,"['{""endDate"": ""2018-07"", ""unified"": true, ""deg...","['{""endDate"": ""2022-03"", ""companyName"": ""奥园集团佛...","['{""endDate"": ""2022-03"", ""description"": ""工作内容:...",20220831,-1,2,4.0,2022-07
82320,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,JD20220725190337904313,产品经理,NaN,110100,产品经理,2c9207158114fb29018132ace5f11fe9,广州视源电子科技股份有限公司,1,...,3.0,NaN,"['{""schoolTag"": ""211"", ""endDate"": ""2019-06"", ""...","['{""endDate"": ""9999-01"", ""companyName"": ""美的"", ...","['{""name"": null}']",20220831,2,2,4.0,2022-07


In [12]:
# 这些特征在原始的cv和jd表里
raw_columns = ['age', 'recruit_number', 'workYear', 'min_month_salary', 'max_month_salary', 'pay_months']

# 将raw特征保存
train_data_lsa_cvjd_54339[['cv_id', 'jd_id'] + raw_columns].to_pickle('../generalization_data/train_raw_feature_54339.pkl')


# ext特征

In [13]:
# 这些特征在sample pipeline的表里
ext_columns = ['school_labels_cv', 'school_labels_jd', 'expectlocation_range_cv', 'location_range_jd', 'degree_index_cv', 'degree_index_jd', 'workyear_range_jd', 'gender_index_cv', 'job_intentstatus_cv']


In [14]:
# 读取包含label的sample表
all_sample = pd.read_csv('../data_20220831/sample_20220831_spark.csv').drop('Unnamed: 0', axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first').reset_index(drop=True)

train_path = '../generalization_data/cvjd_train_filter_54339.pkl'
train_id = pd.read_pickle(train_path)[['cv_id', 'jd_id', 'order_month']]

all_sample = all_sample.join(train_id.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
all_sample

/Users/jinrirencai/opt/anaconda3/envs/tsinghua/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cv_id,jd_id,cv_code,jd_code,last_opt,da_label,rank_label,title,deadline,category_id,...,industry_jd2vec,industry_vec2,majorName_cut,majorName_jd2vec,majorName_vec2,skills_cut,skills_jd2vec,skills_vec2,ds,order_month
0,2c9207157bcb9549017bce3ea49b00fa,2c9207157bcbb76b017bce3d99ba059a,C972256920524165184,JD20210910134407574550,drop,1,1,市场策划总监,2021-11-30 23:59:59.0,140404,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....","['舞蹈', '学']","[0.2290763258934021, -0.14894524216651917, 0.0...","[-0.033116504549980164, -0.058041930198669434,...","['[', '""', 'Ai', '""', ',', '""', 'Ar', '""', ','...","[0.37405160069465637, 0.08108291774988174, -0....","[0.18812726438045502, -0.014001657254993916, -...",20220831,2021-09
2,2c9207157bf91042017c10f871d309f4,2c9207157f347800017f624770866105,C972256920545136700,JD20220216104801688663,drop,1,1,医学博士投资人,NaN,180101,...,"[0.022260336205363274, -0.03374471887946129, -...","[0.30257120728492737, -0.31254786252975464, -0...","['生物医学', '工程']","[0.01792692393064499, -0.1300506740808487, -0....","[0.005150213837623596, 0.15490269660949707, -0...","['[', '""', 'VC', '""', ',', '""', 'PPT', '""', ',...","[0.3084435760974884, 0.07542910426855087, -0.2...","[0.09389838576316833, 0.04999205097556114, -0....",20220831,2022-03
4,2c9207157bf91042017c6ecc163d3aa8,2c9207157bf94d99017c6ecb294930b1,C972256920557719638,JD20211011173942648108,accept,3,3,咨询总监,2021-10-31 23:59:59.0,260401,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",['金融学'],"[0.11331989616155624, -0.1490677297115326, -0....","[-0.3050949275493622, -0.4833742380142212, -2....","['[', '""', 'PPT', '""', ',', '""', 'Word', '""', ...","[0.4078637957572937, 0.07882367819547653, -0.2...","[0.1595805436372757, -0.0552152618765831, -0.3...",20220831,2021-10
5,2c9207157c9f1919017cc065bd545ef8,2c9207157ccbcd86017d08622b3e37f7,C972256921635655974,JD20211110134419018990,reject,2,2,SQE,NaN,120000,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....","['机械设计', '制造', '机器', '自动化']","[-0.021435901522636414, -0.11707526445388794, ...","[-0.05890382081270218, 0.2601788341999054, -0....","['[', '""', 'Pmp', '""', ',', '""', 'Kpi', '""', '...","[0.32037466764450073, 0.12840721011161804, -0....","[0.03882326930761337, 0.061981942504644394, -0...",20220831,2021-11
6,2c9207157c9f1919017cc67653a96ab4,2c9207157f347800017f728750b050e5,C972256920561913876,JD20220215163251463639,drop,1,1,汽车制造成本控制总监,NaN,150310,...,"[0.1326550543308258, -0.17135989665985107, 0.0...","[0.19803519546985626, 0.6821521520614624, -0.2...",['工商管理'],"[0.2706994414329529, -0.26152682304382324, 0.0...","[0.32410022616386414, -1.0074623823165894, -0....","['[', '""', '招聘', '""', ',', '""', '优化', '""', ','...","[0.4110715091228485, 0.16865599155426025, -0.2...","[0.22214311361312866, -0.13587172329425812, -0...",20220831,2022-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82321,8a69d7a68219933d01822594583b5f3d,2c92071581ebf3390182007e0fe74079,C979274582945042432,JD20220715141032849445,reject,2,2,数据分析师,NaN,100511,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",['大连理工大学'],"[0.8574191927909851, 0.08253983408212662, 0.59...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","['Ai', ',', 'PPT', ',', 'SQL', ',', 'Word', ',...","[0.5263129472732544, -0.1190357357263565, -0.0...","[0.30353090167045593, 0.015543319284915924, -0...",20220831,2022-07
82324,8a69d7a68219933d01823d93f8022296,2c92071581ebf3390182009a771b5b07,C1001804828164362240,JD20220705141919254584,drop,1,1,品牌运营中心总经理,NaN,150400,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",['上海交通大学'],"[0.1250215619802475, 0.08120963722467422, -0.1...","[-0.29748210310935974, -0.050827912986278534, ...","['Erp', ',', 'Kpi', ',', 'Page', 

In [15]:
train_id.columns

Index(['cv_id', 'jd_id', 'order_month'], dtype='object')

In [16]:
# 将ext特征保存
all_sample[['cv_id', 'jd_id'] + ext_columns].to_pickle('../generalization_data/train_ext_feature_54339.pkl')

# 拼接new raw ext特征


In [17]:
base_path = '../generalization_data/'
raw_feature_file = 'train_raw_feature_54339.pkl'
ext_feature_file = 'train_ext_feature_54339.pkl'
new_feature_file = 'train_new_feature_54339.pkl'

# 读取特征文件
raw_feature = pd.read_pickle(base_path + raw_feature_file)
ext_feature = pd.read_pickle(base_path + ext_feature_file)
new_feature = pd.read_pickle(base_path + new_feature_file)


In [18]:
new_feature

,cv_id,jd_id,train_description_tfidf_lsa,train_position_tfidf_lsa,train_requirement_tfidf_lsa,train_skills_tfidf_lsa,train_title_category_tags_tfidf_lsa,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[0.2459783180342179, 0.21938831787537966, -0.0...","[0.007319529543557632, 0.0003354154328290771, ...","[0.033013137623400135, -0.035688917703485826, ...","[0.53765937650696, -0.14639534434021456, -0.07...","[0.139618999709323, -0.030086416810889256, -0....",1209.5,3,-1,-1
1,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[0.26393201542159855, 0.10435436209119428, -0....","[0.0995745047074556, -0.02088326554704094, -0....","[0.37018351050902737, -0.20255122674942938, -0...","[0.236061264433608, -0.12857925603594755, 0.13...","[0.5419245911087135, -0.18238753835825924, -0....",76.5,3,-1,-1
2,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.27642059835889154, -0.02735173829637042, 0....","[0.043222795997538296, 0.01108309987508924, 0....","[0.28941341021190825, -0.0474103810249287, -0....","[0.00406004143441403, 0.005098830611803655, 0....","[0.09166404085822458, 0.013460082203324779, -0...",1034.0,1,450,-1
3,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,"[0.20328670858820733, -0.005197850015064169, -...","[0.15026256194834306, -0.004905085247130754, -...","[0.18618173823787507, 0.031168513777247584, 0....","[0.1555934824946681, 0.15537445471422626, -0.0...","[0.08785093521473429, 0.04596513112440933, -0....",NaN,2,-1,-1
4,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,"[0.20498743446953704, -0.006300276619096516, -...","[0.15026256194834306, -0.004905085247130754, -...","[0.19163011557219942, 0.025951225287628978, 0....","[0.1555934824946681, 0.15537445471422626, -0.0...","[0.10718268355356483, 0.05565724707601688, -0....",NaN,2,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...
54334,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,"[0.2625502866681783, -0.08516054875935031, 0.0...","[0.23530919072669182, 0.7944767263661332, 0.01...","[0.30282726216339745, 0.16097547637426476, 0.0...","[0.27210694684652126, 0.42441791865396394, -0....","[0.08847069778780742, 0.22734187469570663, -0....",1075.0,5,-1,300
54335,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,"[0.13488778904906526, -0.043533761211661984, 0...","[0.6087412984522569, -0.1473860953415076, -0.0...","[0.15424621070300917, -0.0012759763333448448, ...","[0.2628487548869251, 0.0075222509233238725, -0...","[0.19044431984360052, -0.019572235686217573, 0...",796.5,7,1020,850
54336,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,"[0.192223443587156, 0.25067413083354817, 0.053...","[0.024225991980613454, -0.0015968824512811737,...","[0.23573037593299082, -0.13812537376984757, 0....","[0.3526478953565766, -0.20899689717605643, 0.1...","[0.1171332341680559, -0.006169459729397869, -0...",1019.0,1,600,120
54337,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,"[0.2550351130980516, -0.09468529288217858, -0....","[0.8678408186646723, -0.22318129673531545, -0....","[0.19019357171237353, -0.12296867979940661, -0...","[0.27948075994260324, -0.04971016234222427, 0....","[0.3466942038817421, -0.07639258814853622, 0.3...",NaN,4,280,240


In [19]:
# 3份特征文件拼接
medium = raw_feature.join(ext_feature.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
raw_ext_new_feature = medium.join(new_feature.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
raw_ext_new_feature

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,job_intentstatus_cv,train_description_tfidf_lsa,train_position_tfidf_lsa,train_requirement_tfidf_lsa,train_skills_tfidf_lsa,train_title_category_tags_tfidf_lsa,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,36.0,1,13.0,NaN,NaN,NaN,[],[],...,3,"[0.2459783180342179, 0.21938831787537966, -0.0...","[0.007319529543557632, 0.0003354154328290771, ...","[0.033013137623400135, -0.035688917703485826, ...","[0.53765937650696, -0.14639534434021456, -0.07...","[0.139618999709323, -0.030086416810889256, -0....",1209.5,3,-1,-1
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,27.0,1,3.0,NaN,NaN,NaN,"['211', '双一流']",[],...,3,"[0.26393201542159855, 0.10435436209119428, -0....","[0.0995745047074556, -0.02088326554704094, -0....","[0.37018351050902737, -0.20255122674942938, -0...","[0.236061264433608, -0.12857925603594755, 0.13...","[0.5419245911087135, -0.18238753835825924, -0....",76.5,3,-1,-1
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,28.0,1,5.0,25.0,45.0,12.0,"['211', '双一流']",[],...,3,"[0.27642059835889154, -0.02735173829637042, 0....","[0.043222795997538296, 0.01108309987508924, 0....","[0.28941341021190825, -0.0474103810249287, -0....","[0.00406004143441403, 0.005098830611803655, 0....","[0.09166404085822458, 0.013460082203324779, -0...",1034.0,1,450,-1
5,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,38.0,1,11.0,30.0,40.0,15.0,"['985', '211', '双一流']","['211', '985', '双一流', 'qs100']",...,3,"[0.20328670858820733, -0.005197850015064169, -...","[0.15026256194834306, -0.004905085247130754, -...","[0.18618173823787507, 0.031168513777247584, 0....","[0.1555934824946681, 0.15537445471422626, -0.0...","[0.08785093521473429, 0.04596513112440933, -0....",NaN,2,-1,-1
6,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,38.0,2,11.0,40.0,60.0,20.0,"['985', '211', '双一流']","['双一流', '985', '211']",...,3,"[0.20498743446953704, -0.006300276619096516, -...","[0.15026256194834306, -0.004905085247130754, -...","[0.19163011557219942, 0.025951225287628978, 0....","[0.1555934824946681, 0.15537445471422626, -0.0...","[0.10718268355356483, 0.05565724707601688, -0....",NaN,2,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82315,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,29.0,1,9.0,25.0,50.0,15.0,"['211', '双一流']",[],...,0,"[0.2625502866681783, -0.08516054875935031, 0.0...","[0.23530919072669182, 0.7944767263661332, 0.01...","[0.30282726216339745, 0.16097547637426476, 0.0...","[0.27210694684652126, 0.42441791865396394, -0....","[0.08847069778780742, 0.22734187469570663, -0....",1075.0,5,-1,300
82316,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,38.0,1,16.0,25.0,45.0,17.0,[],[],...,2,"[0.13488778904906526, -0.043533761211661984, 0...","[0.6087412984522569, -0.1473860953415076, -0.0...","[0.15424621070300917, -0.0012759763333448448, ...","[0.2628487548869251, 0.0075222509233238725, -0...","[0.19044431984360052, -0.019572235686217573, 0...",796.5,7,1020,850
82317,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,27.0,1,4.0,7.0,10.0,12.0,[],[],...,2,"[0.192223443587156, 0.25067413083354817, 0.053...","[0.024225991980613454, -0.0015968824512811737,...","[0.23573037593299082, -0.13812537376984757, 0....","[0.3526478953565766, -0.20899689717605643, 0.1...","[0.1171332341680559, -0.006169459729397869, -0...",1019.0,1,600,120
82320,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,29.0,1,3.0,20.0,35.0,13.0,"['211', '双一流']",[],...,2,"[0.2550351130980516, -0.09468529288217858, -0....","[0.8678408186646723, -0.22318129673531545, -0....","[0.19019357171237353, -0.12296867979940661, -0...","[0.27948075994260324, -0.04971016234222427, 0....","[0.3466942038817421, -0.0763925881

In [20]:
# 读取包含label的sample表
all_sample = pd.read_csv('../data_20220831/sample_20220831_spark.csv').drop('Unnamed: 0', axis=1).drop_duplicates(subset=['cv_id', 'jd_id'], keep='first').reset_index(drop=True)

train_path = '../generalization_data/cvjd_train_filter_54339.pkl'
train_id = pd.read_pickle(train_path)[['cv_id', 'jd_id', 'order_month']]

all_sample = all_sample.join(train_id.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
all_sample

/Users/jinrirencai/opt/anaconda3/envs/tsinghua/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cv_id,jd_id,cv_code,jd_code,last_opt,da_label,rank_label,title,deadline,category_id,...,industry_jd2vec,industry_vec2,majorName_cut,majorName_jd2vec,majorName_vec2,skills_cut,skills_jd2vec,skills_vec2,ds,order_month
0,2c9207157bcb9549017bce3ea49b00fa,2c9207157bcbb76b017bce3d99ba059a,C972256920524165184,JD20210910134407574550,drop,1,1,市场策划总监,2021-11-30 23:59:59.0,140404,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....","['舞蹈', '学']","[0.2290763258934021, -0.14894524216651917, 0.0...","[-0.033116504549980164, -0.058041930198669434,...","['[', '""', 'Ai', '""', ',', '""', 'Ar', '""', ','...","[0.37405160069465637, 0.08108291774988174, -0....","[0.18812726438045502, -0.014001657254993916, -...",20220831,2021-09
2,2c9207157bf91042017c10f871d309f4,2c9207157f347800017f624770866105,C972256920545136700,JD20220216104801688663,drop,1,1,医学博士投资人,NaN,180101,...,"[0.022260336205363274, -0.03374471887946129, -...","[0.30257120728492737, -0.31254786252975464, -0...","['生物医学', '工程']","[0.01792692393064499, -0.1300506740808487, -0....","[0.005150213837623596, 0.15490269660949707, -0...","['[', '""', 'VC', '""', ',', '""', 'PPT', '""', ',...","[0.3084435760974884, 0.07542910426855087, -0.2...","[0.09389838576316833, 0.04999205097556114, -0....",20220831,2022-03
4,2c9207157bf91042017c6ecc163d3aa8,2c9207157bf94d99017c6ecb294930b1,C972256920557719638,JD20211011173942648108,accept,3,3,咨询总监,2021-10-31 23:59:59.0,260401,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",['金融学'],"[0.11331989616155624, -0.1490677297115326, -0....","[-0.3050949275493622, -0.4833742380142212, -2....","['[', '""', 'PPT', '""', ',', '""', 'Word', '""', ...","[0.4078637957572937, 0.07882367819547653, -0.2...","[0.1595805436372757, -0.0552152618765831, -0.3...",20220831,2021-10
5,2c9207157c9f1919017cc065bd545ef8,2c9207157ccbcd86017d08622b3e37f7,C972256921635655974,JD20211110134419018990,reject,2,2,SQE,NaN,120000,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....","['机械设计', '制造', '机器', '自动化']","[-0.021435901522636414, -0.11707526445388794, ...","[-0.05890382081270218, 0.2601788341999054, -0....","['[', '""', 'Pmp', '""', ',', '""', 'Kpi', '""', '...","[0.32037466764450073, 0.12840721011161804, -0....","[0.03882326930761337, 0.061981942504644394, -0...",20220831,2021-11
6,2c9207157c9f1919017cc67653a96ab4,2c9207157f347800017f728750b050e5,C972256920561913876,JD20220215163251463639,drop,1,1,汽车制造成本控制总监,NaN,150310,...,"[0.1326550543308258, -0.17135989665985107, 0.0...","[0.19803519546985626, 0.6821521520614624, -0.2...",['工商管理'],"[0.2706994414329529, -0.26152682304382324, 0.0...","[0.32410022616386414, -1.0074623823165894, -0....","['[', '""', '招聘', '""', ',', '""', '优化', '""', ','...","[0.4110715091228485, 0.16865599155426025, -0.2...","[0.22214311361312866, -0.13587172329425812, -0...",20220831,2022-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82321,8a69d7a68219933d01822594583b5f3d,2c92071581ebf3390182007e0fe74079,C979274582945042432,JD20220715141032849445,reject,2,2,数据分析师,NaN,100511,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",['大连理工大学'],"[0.8574191927909851, 0.08253983408212662, 0.59...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","['Ai', ',', 'PPT', ',', 'SQL', ',', 'Word', ',...","[0.5263129472732544, -0.1190357357263565, -0.0...","[0.30353090167045593, 0.015543319284915924, -0...",20220831,2022-07
82324,8a69d7a68219933d01823d93f8022296,2c92071581ebf3390182009a771b5b07,C1001804828164362240,JD20220705141919254584,drop,1,1,品牌运营中心总经理,NaN,150400,...,"[-0.009751920588314533, 0.17647229135036469, -...","[0.20024815201759338, 0.30229872465133667, -0....",['上海交通大学'],"[0.1250215619802475, 0.08120963722467422, -0.1...","[-0.29748210310935974, -0.050827912986278534, ...","['Erp', ',', 'Kpi', ',', 'Page', 

In [21]:
# 拼接特征和label并保存
label_df = all_sample[['cv_id', 'jd_id', 'da_label', 'rank_label']]
raw_ext_new_feature_label = raw_ext_new_feature.join(label_df.set_index(['cv_id', 'jd_id']), on=['cv_id', 'jd_id'], how='inner')
raw_ext_new_feature_label.to_pickle('../generalization_data/train_raw_ext_new_feature_label_54339.pkl')

In [22]:
raw_ext_new_feature_label

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,train_position_tfidf_lsa,train_requirement_tfidf_lsa,train_skills_tfidf_lsa,train_title_category_tags_tfidf_lsa,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary,da_label,rank_label
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,36.0,1,13.0,NaN,NaN,NaN,[],[],...,"[0.007319529543557632, 0.0003354154328290771, ...","[0.033013137623400135, -0.035688917703485826, ...","[0.53765937650696, -0.14639534434021456, -0.07...","[0.139618999709323, -0.030086416810889256, -0....",1209.5,3,-1,-1,3,3
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,27.0,1,3.0,NaN,NaN,NaN,"['211', '双一流']",[],...,"[0.0995745047074556, -0.02088326554704094, -0....","[0.37018351050902737, -0.20255122674942938, -0...","[0.236061264433608, -0.12857925603594755, 0.13...","[0.5419245911087135, -0.18238753835825924, -0....",76.5,3,-1,-1,-1,2
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,28.0,1,5.0,25.0,45.0,12.0,"['211', '双一流']",[],...,"[0.043222795997538296, 0.01108309987508924, 0....","[0.28941341021190825, -0.0474103810249287, -0....","[0.00406004143441403, 0.005098830611803655, 0....","[0.09166404085822458, 0.013460082203324779, -0...",1034.0,1,450,-1,1,1
5,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,38.0,1,11.0,30.0,40.0,15.0,"['985', '211', '双一流']","['211', '985', '双一流', 'qs100']",...,"[0.15026256194834306, -0.004905085247130754, -...","[0.18618173823787507, 0.031168513777247584, 0....","[0.1555934824946681, 0.15537445471422626, -0.0...","[0.08785093521473429, 0.04596513112440933, -0....",NaN,2,-1,-1,2,2
6,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,38.0,2,11.0,40.0,60.0,20.0,"['985', '211', '双一流']","['双一流', '985', '211']",...,"[0.15026256194834306, -0.004905085247130754, -...","[0.19163011557219942, 0.025951225287628978, 0....","[0.1555934824946681, 0.15537445471422626, -0.0...","[0.10718268355356483, 0.05565724707601688, -0....",NaN,2,-1,-1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82315,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,29.0,1,9.0,25.0,50.0,15.0,"['211', '双一流']",[],...,"[0.23530919072669182, 0.7944767263661332, 0.01...","[0.30282726216339745, 0.16097547637426476, 0.0...","[0.27210694684652126, 0.42441791865396394, -0....","[0.08847069778780742, 0.22734187469570663, -0....",1075.0,5,-1,300,2,2
82316,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,38.0,1,16.0,25.0,45.0,17.0,[],[],...,"[0.6087412984522569, -0.1473860953415076, -0.0...","[0.15424621070300917, -0.0012759763333448448, ...","[0.2628487548869251, 0.0075222509233238725, -0...","[0.19044431984360052, -0.019572235686217573, 0...",796.5,7,1020,850,1,1
82317,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,27.0,1,4.0,7.0,10.0,12.0,[],[],...,"[0.024225991980613454, -0.0015968824512811737,...","[0.23573037593299082, -0.13812537376984757, 0....","[0.3526478953565766, -0.20899689717605643, 0.1...","[0.1171332341680559, -0.006169459729397869, -0...",1019.0,1,600,120,-1,2
82320,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,29.0,1,3.0,20.0,35.0,13.0,"['211', '双一流']",[],...,"[0.8678408186646723, -0.22318129673531545, -0....","[0.19019357171237353, -0.12296867979940661, -0...","[0.27948075994260324, -0.04971016234222427, 0....","[0.3466942038817421, -0.07639258814853622, 0.3...",NaN,4,280,240,2,2


In [23]:
sample_train = raw_ext_new_feature_label

In [24]:
sample_train.columns

Index(['cv_id', 'jd_id', 'age', 'recruit_number', 'workYear',
       'min_month_salary', 'max_month_salary', 'pay_months',
       'school_labels_cv', 'school_labels_jd', 'expectlocation_range_cv',
       'location_range_jd', 'degree_index_cv', 'degree_index_jd',
       'workyear_range_jd', 'gender_index_cv', 'job_intentstatus_cv',
       'train_description_tfidf_lsa', 'train_position_tfidf_lsa',
       'train_requirement_tfidf_lsa', 'train_skills_tfidf_lsa',
       'train_title_category_tags_tfidf_lsa', 'work_duration_mean',
       'equal_job', 'parsed_desiredSalary', 'parsed_currentSalary', 'da_label',
       'rank_label'],
      dtype='object')

In [25]:
!pwd

/Users/jinrirencai/Desktop/golden_data_from_github/generalization


In [26]:
# 中位数填充
from sklearn.impute import SimpleImputer
import pickle

imp = SimpleImputer(missing_values=np.nan, strategy='median')

sample_train['age'] = imp.fit_transform(sample_train[['age']])
imp_pkl_filename = 'age_missing_model.pkl'
with open(imp_pkl_filename, 'wb') as file:
    pickle.dump(imp, file)

sample_train['workYear'] = imp.fit_transform(sample_train[['workYear']])
imp_pkl_filename = 'workYear_missing_model.pkl'
with open(imp_pkl_filename, 'wb') as file:
    pickle.dump(imp, file)

sample_train['min_month_salary'] = imp.fit_transform(sample_train[['min_month_salary']])
imp_pkl_filename = 'min_month_salary_missing_model.pkl'
with open(imp_pkl_filename, 'wb') as file:
    pickle.dump(imp, file)

sample_train['max_month_salary'] = imp.fit_transform(sample_train[['max_month_salary']])
imp_pkl_filename = 'max_month_salary_missing_model.pkl'
with open(imp_pkl_filename, 'wb') as file:
    pickle.dump(imp, file)

sample_train['pay_months'] = imp.fit_transform(sample_train[['pay_months']])
imp_pkl_filename = 'pay_months_missing_model.pkl'
with open(imp_pkl_filename, 'wb') as file:
    pickle.dump(imp, file)

sample_train['work_duration_mean'] = imp.fit_transform(sample_train[['work_duration_mean']])
imp_pkl_filename = 'work_duration_mean_missing_model.pkl'
with open(imp_pkl_filename, 'wb') as file:
    pickle.dump(imp, file)




imp = SimpleImputer(missing_values=-1, strategy='median')
# 这两个只能选一个接入模型
sample_train['parsed_desiredSalary'] = imp.fit_transform(sample_train[['parsed_desiredSalary']])
imp_pkl_filename = 'parsed_desiredSalary_missing_model.pkl'
with open(imp_pkl_filename, 'wb') as file:
    pickle.dump(imp, file)

sample_train['parsed_currentSalary'] = imp.fit_transform(sample_train[['parsed_currentSalary']])
imp_pkl_filename = 'parsed_currentSalary_missing_model.pkl'
with open(imp_pkl_filename, 'wb') as file:
    pickle.dump(imp, file)

sample_train


,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,train_position_tfidf_lsa,train_requirement_tfidf_lsa,train_skills_tfidf_lsa,train_title_category_tags_tfidf_lsa,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary,da_label,rank_label
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,36.0,1,13.0,25.0,40.0,14.0,[],[],...,"[0.007319529543557632, 0.0003354154328290771, ...","[0.033013137623400135, -0.035688917703485826, ...","[0.53765937650696, -0.14639534434021456, -0.07...","[0.139618999709323, -0.030086416810889256, -0....",1209.5,3,360.0,300.0,3,3
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,27.0,1,3.0,25.0,40.0,14.0,"['211', '双一流']",[],...,"[0.0995745047074556, -0.02088326554704094, -0....","[0.37018351050902737, -0.20255122674942938, -0...","[0.236061264433608, -0.12857925603594755, 0.13...","[0.5419245911087135, -0.18238753835825924, -0....",76.5,3,360.0,300.0,-1,2
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,28.0,1,5.0,25.0,45.0,12.0,"['211', '双一流']",[],...,"[0.043222795997538296, 0.01108309987508924, 0....","[0.28941341021190825, -0.0474103810249287, -0....","[0.00406004143441403, 0.005098830611803655, 0....","[0.09166404085822458, 0.013460082203324779, -0...",1034.0,1,450.0,300.0,1,1
5,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,38.0,1,11.0,30.0,40.0,15.0,"['985', '211', '双一流']","['211', '985', '双一流', 'qs100']",...,"[0.15026256194834306, -0.004905085247130754, -...","[0.18618173823787507, 0.031168513777247584, 0....","[0.1555934824946681, 0.15537445471422626, -0.0...","[0.08785093521473429, 0.04596513112440933, -0....",776.5,2,360.0,300.0,2,2
6,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,38.0,2,11.0,40.0,60.0,20.0,"['985', '211', '双一流']","['双一流', '985', '211']",...,"[0.15026256194834306, -0.004905085247130754, -...","[0.19163011557219942, 0.025951225287628978, 0....","[0.1555934824946681, 0.15537445471422626, -0.0...","[0.10718268355356483, 0.05565724707601688, -0....",776.5,2,360.0,300.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82315,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,29.0,1,9.0,25.0,50.0,15.0,"['211', '双一流']",[],...,"[0.23530919072669182, 0.7944767263661332, 0.01...","[0.30282726216339745, 0.16097547637426476, 0.0...","[0.27210694684652126, 0.42441791865396394, -0....","[0.08847069778780742, 0.22734187469570663, -0....",1075.0,5,360.0,300.0,2,2
82316,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,38.0,1,16.0,25.0,45.0,17.0,[],[],...,"[0.6087412984522569, -0.1473860953415076, -0.0...","[0.15424621070300917, -0.0012759763333448448, ...","[0.2628487548869251, 0.0075222509233238725, -0...","[0.19044431984360052, -0.019572235686217573, 0...",796.5,7,1020.0,850.0,1,1
82317,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,27.0,1,4.0,7.0,10.0,12.0,[],[],...,"[0.024225991980613454, -0.0015968824512811737,...","[0.23573037593299082, -0.13812537376984757, 0....","[0.3526478953565766, -0.20899689717605643, 0.1...","[0.1171332341680559, -0.006169459729397869, -0...",1019.0,1,600.0,120.0,-1,2
82320,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,29.0,1,3.0,20.0,35.0,13.0,"['211', '双一流']",[],...,"[0.8678408186646723, -0.22318129673531545, -0....","[0.19019357171237353, -0.12296867979940661, -0...","[0.27948075994260324, -0.04971016234222427, 0....","[0.3466942038817421, -0.07639258814853622, 0.3...",776.5,4,280.0,240.0,2,2


# 手动cross

In [27]:
# 首先把school_labels_cv和school_labels_jd转成列表
def school_lables_to_list(series, school_col):
    school_labels = series[school_col]
    school_labels_list = [item.strip('\'') for item in school_labels.strip('[]').replace(' ', '').split(',')]
    if school_labels_list[0] == '':
        return []
    return school_labels_list
sample_train['school_labels_cv_list'] = sample_train.apply(school_lables_to_list, axis=1, args=('school_labels_cv', ))
sample_train['school_labels_jd_list'] = sample_train.apply(school_lables_to_list, axis=1, args=('school_labels_jd', ))

sample_train

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,train_skills_tfidf_lsa,train_title_category_tags_tfidf_lsa,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary,da_label,rank_label,school_labels_cv_list,school_labels_jd_list
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,36.0,1,13.0,25.0,40.0,14.0,[],[],...,"[0.53765937650696, -0.14639534434021456, -0.07...","[0.139618999709323, -0.030086416810889256, -0....",1209.5,3,360.0,300.0,3,3,[],[]
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,27.0,1,3.0,25.0,40.0,14.0,"['211', '双一流']",[],...,"[0.236061264433608, -0.12857925603594755, 0.13...","[0.5419245911087135, -0.18238753835825924, -0....",76.5,3,360.0,300.0,-1,2,"[211, 双一流]",[]
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,28.0,1,5.0,25.0,45.0,12.0,"['211', '双一流']",[],...,"[0.00406004143441403, 0.005098830611803655, 0....","[0.09166404085822458, 0.013460082203324779, -0...",1034.0,1,450.0,300.0,1,1,"[211, 双一流]",[]
5,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,38.0,1,11.0,30.0,40.0,15.0,"['985', '211', '双一流']","['211', '985', '双一流', 'qs100']",...,"[0.1555934824946681, 0.15537445471422626, -0.0...","[0.08785093521473429, 0.04596513112440933, -0....",776.5,2,360.0,300.0,2,2,"[985, 211, 双一流]","[211, 985, 双一流, qs100]"
6,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,38.0,2,11.0,40.0,60.0,20.0,"['985', '211', '双一流']","['双一流', '985', '211']",...,"[0.1555934824946681, 0.15537445471422626, -0.0...","[0.10718268355356483, 0.05565724707601688, -0....",776.5,2,360.0,300.0,1,1,"[985, 211, 双一流]","[双一流, 985, 211]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82315,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,29.0,1,9.0,25.0,50.0,15.0,"['211', '双一流']",[],...,"[0.27210694684652126, 0.42441791865396394, -0....","[0.08847069778780742, 0.22734187469570663, -0....",1075.0,5,360.0,300.0,2,2,"[211, 双一流]",[]
82316,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,38.0,1,16.0,25.0,45.0,17.0,[],[],...,"[0.2628487548869251, 0.0075222509233238725, -0...","[0.19044431984360052, -0.019572235686217573, 0...",796.5,7,1020.0,850.0,1,1,[],[]
82317,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,27.0,1,4.0,7.0,10.0,12.0,[],[],...,"[0.3526478953565766, -0.20899689717605643, 0.1...","[0.1171332341680559, -0.006169459729397869, -0...",1019.0,1,600.0,120.0,-1,2,[],[]
82320,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,29.0,1,3.0,20.0,35.0,13.0,"['211', '双一流']",[],...,"[0.27948075994260324, -0.04971016234222427, 0....","[0.3466942038817421, -0.07639258814853622, 0.3...",776.5,4,280.0,240.0,2,2,"[211, 双一流]",[]


In [28]:
# school等级交叉
def school_intersect_cvjd(series, school_labels_cv, school_labels_jd):
    school_labels_cv = series[school_labels_cv]
    school_labels_jd = series[school_labels_jd]
    # jd没有学历要求or解析不出来
    if not school_labels_jd:
        return 1
    if set(school_labels_cv).intersection(set(school_labels_jd)):
        return 1
    return 0

sample_train['school_intersect_cvjd'] = sample_train.apply(school_intersect_cvjd, axis=1, args=('school_labels_cv_list', 'school_labels_jd_list'))
sample_train

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,train_title_category_tags_tfidf_lsa,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary,da_label,rank_label,school_labels_cv_list,school_labels_jd_list,school_intersect_cvjd
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,36.0,1,13.0,25.0,40.0,14.0,[],[],...,"[0.139618999709323, -0.030086416810889256, -0....",1209.5,3,360.0,300.0,3,3,[],[],1
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,27.0,1,3.0,25.0,40.0,14.0,"['211', '双一流']",[],...,"[0.5419245911087135, -0.18238753835825924, -0....",76.5,3,360.0,300.0,-1,2,"[211, 双一流]",[],1
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,28.0,1,5.0,25.0,45.0,12.0,"['211', '双一流']",[],...,"[0.09166404085822458, 0.013460082203324779, -0...",1034.0,1,450.0,300.0,1,1,"[211, 双一流]",[],1
5,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,38.0,1,11.0,30.0,40.0,15.0,"['985', '211', '双一流']","['211', '985', '双一流', 'qs100']",...,"[0.08785093521473429, 0.04596513112440933, -0....",776.5,2,360.0,300.0,2,2,"[985, 211, 双一流]","[211, 985, 双一流, qs100]",1
6,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,38.0,2,11.0,40.0,60.0,20.0,"['985', '211', '双一流']","['双一流', '985', '211']",...,"[0.10718268355356483, 0.05565724707601688, -0....",776.5,2,360.0,300.0,1,1,"[985, 211, 双一流]","[双一流, 985, 211]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82315,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,29.0,1,9.0,25.0,50.0,15.0,"['211', '双一流']",[],...,"[0.08847069778780742, 0.22734187469570663, -0....",1075.0,5,360.0,300.0,2,2,"[211, 双一流]",[],1
82316,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,38.0,1,16.0,25.0,45.0,17.0,[],[],...,"[0.19044431984360052, -0.019572235686217573, 0...",796.5,7,1020.0,850.0,1,1,[],[],1
82317,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,27.0,1,4.0,7.0,10.0,12.0,[],[],...,"[0.1171332341680559, -0.006169459729397869, -0...",1019.0,1,600.0,120.0,-1,2,[],[],1
82320,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,29.0,1,3.0,20.0,35.0,13.0,"['211', '双一流']",[],...,"[0.3466942038817421, -0.07639258814853622, 0.3...",776.5,4,280.0,240.0,2,2,"[211, 双一流]",[],1


In [29]:
# 首先把expectlocation_range_cv和location_range_jd转成列表
def school_lables_to_list(series, location_col):
    location_labels = series[location_col]
    location_labels_list = [item.strip('\'') for item in location_labels.strip('[]').replace(' ', '').split(',')]
    if location_labels_list[0] == '':
        return []
    return location_labels_list

sample_train['location_range_jd_list'] = sample_train.apply(school_lables_to_list, axis=1, args=('location_range_jd', ))
sample_train['expectlocation_range_cv_list'] = sample_train.apply(school_lables_to_list, axis=1, args=('expectlocation_range_cv', ))

sample_train

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,equal_job,parsed_desiredSalary,parsed_currentSalary,da_label,rank_label,school_labels_cv_list,school_labels_jd_list,school_intersect_cvjd,location_range_jd_list,expectlocation_range_cv_list
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,36.0,1,13.0,25.0,40.0,14.0,[],[],...,3,360.0,300.0,3,3,[],[],1,[深圳市],[未知市]
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,27.0,1,3.0,25.0,40.0,14.0,"['211', '双一流']",[],...,3,360.0,300.0,-1,2,"[211, 双一流]",[],1,[广州市],[未知市]
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,28.0,1,5.0,25.0,45.0,12.0,"['211', '双一流']",[],...,1,450.0,300.0,1,1,"[211, 双一流]",[],1,[],[深圳市]
5,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,38.0,1,11.0,30.0,40.0,15.0,"['985', '211', '双一流']","['211', '985', '双一流', 'qs100']",...,2,360.0,300.0,2,2,"[985, 211, 双一流]","[211, 985, 双一流, qs100]",1,[],[未知市]
6,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,38.0,2,11.0,40.0,60.0,20.0,"['985', '211', '双一流']","['双一流', '985', '211']",...,2,360.0,300.0,1,1,"[985, 211, 双一流]","[双一流, 985, 211]",1,[],[未知市]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82315,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,29.0,1,9.0,25.0,50.0,15.0,"['211', '双一流']",[],...,5,360.0,300.0,2,2,"[211, 双一流]",[],1,[],[北京市]
82316,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,38.0,1,16.0,25.0,45.0,17.0,[],[],...,7,1020.0,850.0,1,1,[],[],1,[],[深圳市]
82317,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,27.0,1,4.0,7.0,10.0,12.0,[],[],...,1,600.0,120.0,-1,2,[],[],1,[],[未知市]
82320,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,29.0,1,3.0,20.0,35.0,13.0,"['211', '双一流']",[],...,4,280.0,240.0,2,2,"[211, 双一流]",[],1,[],[广州市]


In [30]:
# location交叉
def location_intersect_cvjd(series, location_labels_cv, location_labels_jd):
    location_labels_cv = series[location_labels_cv]
    location_labels_jd = series[location_labels_jd]
    # 如果jd没有地域
    if not location_labels_jd:
        return 1
    if set(location_labels_cv).intersection(set(location_labels_jd)):
        return 1
    return 0

sample_train['location_intersect_cvjd'] = sample_train.apply(location_intersect_cvjd, axis=1, args=('expectlocation_range_cv_list', 'location_range_jd_list'))
sample_train

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,parsed_desiredSalary,parsed_currentSalary,da_label,rank_label,school_labels_cv_list,school_labels_jd_list,school_intersect_cvjd,location_range_jd_list,expectlocation_range_cv_list,location_intersect_cvjd
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,36.0,1,13.0,25.0,40.0,14.0,[],[],...,360.0,300.0,3,3,[],[],1,[深圳市],[未知市],0
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,27.0,1,3.0,25.0,40.0,14.0,"['211', '双一流']",[],...,360.0,300.0,-1,2,"[211, 双一流]",[],1,[广州市],[未知市],0
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,28.0,1,5.0,25.0,45.0,12.0,"['211', '双一流']",[],...,450.0,300.0,1,1,"[211, 双一流]",[],1,[],[深圳市],1
5,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,38.0,1,11.0,30.0,40.0,15.0,"['985', '211', '双一流']","['211', '985', '双一流', 'qs100']",...,360.0,300.0,2,2,"[985, 211, 双一流]","[211, 985, 双一流, qs100]",1,[],[未知市],1
6,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,38.0,2,11.0,40.0,60.0,20.0,"['985', '211', '双一流']","['双一流', '985', '211']",...,360.0,300.0,1,1,"[985, 211, 双一流]","[双一流, 985, 211]",1,[],[未知市],1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82315,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,29.0,1,9.0,25.0,50.0,15.0,"['211', '双一流']",[],...,360.0,300.0,2,2,"[211, 双一流]",[],1,[],[北京市],1
82316,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,38.0,1,16.0,25.0,45.0,17.0,[],[],...,1020.0,850.0,1,1,[],[],1,[],[深圳市],1
82317,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,27.0,1,4.0,7.0,10.0,12.0,[],[],...,600.0,120.0,-1,2,[],[],1,[],[未知市],1
82320,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,29.0,1,3.0,20.0,35.0,13.0,"['211', '双一流']",[],...,280.0,240.0,2,2,"[211, 双一流]",[],1,[],[广州市],1


In [31]:
# degree交叉
def degree_match_cvjd(series, degree_index_cv, degree_index_jd):
    degree_index_cv = series['degree_index_cv']
    degree_index_jd = series['degree_index_jd']
    if degree_index_cv >= degree_index_jd:
        return 1
    else:
        return 0

sample_train['degree_match_cvjd'] = sample_train.apply(degree_match_cvjd, axis=1, args=('degree_index_cv', 'degree_index_jd'))
sample_train

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,parsed_currentSalary,da_label,rank_label,school_labels_cv_list,school_labels_jd_list,school_intersect_cvjd,location_range_jd_list,expectlocation_range_cv_list,location_intersect_cvjd,degree_match_cvjd
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,36.0,1,13.0,25.0,40.0,14.0,[],[],...,300.0,3,3,[],[],1,[深圳市],[未知市],0,1
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,27.0,1,3.0,25.0,40.0,14.0,"['211', '双一流']",[],...,300.0,-1,2,"[211, 双一流]",[],1,[广州市],[未知市],0,1
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,28.0,1,5.0,25.0,45.0,12.0,"['211', '双一流']",[],...,300.0,1,1,"[211, 双一流]",[],1,[],[深圳市],1,1
5,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,38.0,1,11.0,30.0,40.0,15.0,"['985', '211', '双一流']","['211', '985', '双一流', 'qs100']",...,300.0,2,2,"[985, 211, 双一流]","[211, 985, 双一流, qs100]",1,[],[未知市],1,1
6,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,38.0,2,11.0,40.0,60.0,20.0,"['985', '211', '双一流']","['双一流', '985', '211']",...,300.0,1,1,"[985, 211, 双一流]","[双一流, 985, 211]",1,[],[未知市],1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82315,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,29.0,1,9.0,25.0,50.0,15.0,"['211', '双一流']",[],...,300.0,2,2,"[211, 双一流]",[],1,[],[北京市],1,1
82316,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,38.0,1,16.0,25.0,45.0,17.0,[],[],...,850.0,1,1,[],[],1,[],[深圳市],1,1
82317,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,27.0,1,4.0,7.0,10.0,12.0,[],[],...,120.0,-1,2,[],[],1,[],[未知市],1,1
82320,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,29.0,1,3.0,20.0,35.0,13.0,"['211', '双一流']",[],...,240.0,2,2,"[211, 双一流]",[],1,[],[广州市],1,1


In [32]:
sample_train['rank_label'].value_counts()

1    18963
2    15910
0    12984
3     6482
Name: rank_label, dtype: int64

In [33]:
# 生成模型训练label
def model_label_gen(series, rank_label):
    rank_label = series[rank_label]
    if rank_label in [0, 1]:
        return 0
    return 1
sample_train['model_label'] = sample_train.apply(model_label_gen, axis=1, args=('rank_label', ))
sample_train

,cv_id,jd_id,age,recruit_number,workYear,min_month_salary,max_month_salary,pay_months,school_labels_cv,school_labels_jd,...,da_label,rank_label,school_labels_cv_list,school_labels_jd_list,school_intersect_cvjd,location_range_jd_list,expectlocation_range_cv_list,location_intersect_cvjd,degree_match_cvjd,model_label
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,36.0,1,13.0,25.0,40.0,14.0,[],[],...,3,3,[],[],1,[深圳市],[未知市],0,1,1
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,27.0,1,3.0,25.0,40.0,14.0,"['211', '双一流']",[],...,-1,2,"[211, 双一流]",[],1,[广州市],[未知市],0,1,1
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,28.0,1,5.0,25.0,45.0,12.0,"['211', '双一流']",[],...,1,1,"[211, 双一流]",[],1,[],[深圳市],1,1,0
5,2c9207157dbedf8d017dc1503c891402,2c9207157dc878e5017de5ee152b646c,38.0,1,11.0,30.0,40.0,15.0,"['985', '211', '双一流']","['211', '985', '双一流', 'qs100']",...,2,2,"[985, 211, 双一流]","[211, 985, 双一流, qs100]",1,[],[未知市],1,1,1
6,2c9207157dbedf8d017dc1503c891402,2c9207157dbedfdb017dc14a85ff432f,38.0,2,11.0,40.0,60.0,20.0,"['985', '211', '双一流']","['双一流', '985', '211']",...,1,1,"[985, 211, 双一流]","[双一流, 985, 211]",1,[],[未知市],1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82315,8a69d7a68219933d018248864f066edd,8a69f601823a92b00182494187c01920,29.0,1,9.0,25.0,50.0,15.0,"['211', '双一流']",[],...,2,2,"[211, 双一流]",[],1,[],[北京市],1,1,1
82316,8a69d7a68219933d01824944cb1533be,8a69c4a8820f0ee00182143637b71ef3,38.0,1,16.0,25.0,45.0,17.0,[],[],...,1,1,[],[],1,[],[深圳市],1,1,0
82317,8a69d7c2821994530182251640042c20,8a69f68e82198e9101822012a308030b,27.0,1,4.0,7.0,10.0,12.0,[],[],...,-1,2,[],[],1,[],[未知市],1,1,1
82320,8a69d7c28219945301823a2058767134,8a69f68e82198e9101823506dafb2080,29.0,1,3.0,20.0,35.0,13.0,"['211', '双一流']",[],...,2,2,"[211, 双一流]",[],1,[],[广州市],1,1,1


In [34]:
sample_train['model_label'].value_counts()

0    31947
1    22392
Name: model_label, dtype: int64

In [35]:
# 将jd中的月薪转成年薪
def multiply_cols_fun(series, col1, col2):
    col1_value = series[col1]
    col2_value = series[col2]
    return col1_value * col2_value

sample_train['min_annual_salary'] = sample_train.apply(multiply_cols_fun, axis=1, args=('min_month_salary', 'pay_months'))
sample_train['max_annual_salary'] = sample_train.apply(multiply_cols_fun, axis=1, args=('max_month_salary', 'pay_months'))

In [36]:
# 先把workyear_range_jd转成列表
def school_lables_to_list(series, location_col):
    location_labels = series[location_col]
    location_labels_list = [item.strip('\'') for item in location_labels.strip('[]').replace(' ', '').split(',')]
    first = int(location_labels_list[0])
    second = int(location_labels_list[1])
    return [first, second]

sample_train['workyear_range_jd_list'] = sample_train.apply(school_lables_to_list, axis=1, args=('workyear_range_jd', ))

sample_train[['workyear_range_jd', 'workyear_range_jd_list']]

,workyear_range_jd,workyear_range_jd_list
0,"[5, 10]","[5, 10]"
3,"[3, 5]","[3, 5]"
4,"[3, 5]","[3, 5]"
5,"[3, 5]","[3, 5]"
6,"[5, 10]","[5, 10]"
...,...,...
82315,"[3, 5]","[3, 5]"
82316,"[8, 100]","[8, 100]"
82317,"[1, 3]","[1, 3]"
82320,"[5, 10]","[5, 10]"


In [37]:
# work year match
def workYear_match_fun(series, workyear_range_jd_list, workYear):
    workyear_range = series[workyear_range_jd_list]
    workYear = series[workYear]
    if workyear_range[0] <= workYear <= workyear_range[1]:
        return 1
    else:
        return 0
    
sample_train['workyear_match_cvjd'] = sample_train.apply(workYear_match_fun, axis=1, args=('workyear_range_jd_list', 'workYear'))

In [38]:
# 好学校
def keyU_label_fun(series, school_labels_cv_list):
    school_labels = series[school_labels_cv_list]
    if school_labels:
        return 1
    else:
        return 0
sample_train['keyU'] = sample_train.apply(keyU_label_fun, axis=1, args=('school_labels_cv_list',))

In [39]:
sample_train['train_description_tfidf_lsa'][0].__class__

list

In [40]:
sample_train.columns

Index(['cv_id', 'jd_id', 'age', 'recruit_number', 'workYear',
       'min_month_salary', 'max_month_salary', 'pay_months',
       'school_labels_cv', 'school_labels_jd', 'expectlocation_range_cv',
       'location_range_jd', 'degree_index_cv', 'degree_index_jd',
       'workyear_range_jd', 'gender_index_cv', 'job_intentstatus_cv',
       'train_description_tfidf_lsa', 'train_position_tfidf_lsa',
       'train_requirement_tfidf_lsa', 'train_skills_tfidf_lsa',
       'train_title_category_tags_tfidf_lsa', 'work_duration_mean',
       'equal_job', 'parsed_desiredSalary', 'parsed_currentSalary', 'da_label',
       'rank_label', 'school_labels_cv_list', 'school_labels_jd_list',
       'school_intersect_cvjd', 'location_range_jd_list',
       'expectlocation_range_cv_list', 'location_intersect_cvjd',
       'degree_match_cvjd', 'model_label', 'min_annual_salary',
       'max_annual_salary', 'workyear_range_jd_list', 'workyear_match_cvjd',
       'keyU'],
      dtype='object')

In [41]:
# sample train的几个tfidf列重命名
sample_train = sample_train.rename(columns={
                    'train_description_tfidf_lsa': 'description_tfidf_pca',
                    'train_position_tfidf_lsa': 'position_tfidf_pca',
                    'train_requirement_tfidf_lsa': 'requirement_tfidf_pca',
                    'train_skills_tfidf_lsa': 'kills_tfidf_pca',
                    'train_title_category_tags_tfidf_lsa': 'title_category_tags_tfidf_pca' 
})
sample_train.columns

Index(['cv_id', 'jd_id', 'age', 'recruit_number', 'workYear',
       'min_month_salary', 'max_month_salary', 'pay_months',
       'school_labels_cv', 'school_labels_jd', 'expectlocation_range_cv',
       'location_range_jd', 'degree_index_cv', 'degree_index_jd',
       'workyear_range_jd', 'gender_index_cv', 'job_intentstatus_cv',
       'description_tfidf_pca', 'position_tfidf_pca', 'requirement_tfidf_pca',
       'kills_tfidf_pca', 'title_category_tags_tfidf_pca',
       'work_duration_mean', 'equal_job', 'parsed_desiredSalary',
       'parsed_currentSalary', 'da_label', 'rank_label',
       'school_labels_cv_list', 'school_labels_jd_list',
       'school_intersect_cvjd', 'location_range_jd_list',
       'expectlocation_range_cv_list', 'location_intersect_cvjd',
       'degree_match_cvjd', 'model_label', 'min_annual_salary',
       'max_annual_salary', 'workyear_range_jd_list', 'workyear_match_cvjd',
       'keyU'],
      dtype='object')

In [42]:
# 几个list如何处理
# parsed_currentSalary特征不使用
# min_month_salary和max_month_salary分别乘以pay_months，得到年薪数据，以k为单位。已处理
# salary特征可信度不高，清华模型就是简单的输入deep侧，除此以外没有任何处理。
COLUMNS = ['age', 'recruit_number', 'workYear', 'min_annual_salary', 'max_annual_salary', 
            'degree_index_cv', 'degree_index_jd', 'gender_index_cv', 'job_intentstatus_cv',
            'description_tfidf_pca', 'position_tfidf_pca', 'requirement_tfidf_pca', 'kills_tfidf_pca', 'title_category_tags_tfidf_pca',
            'work_duration_mean', 'equal_job', 'parsed_desiredSalary', 'school_intersect_cvjd', 'location_intersect_cvjd',
            'degree_match_cvjd', 'workyear_match_cvjd', 'keyU']

LABEL_COLUMN = "model_label"

# 离散特征
CATEGORICAL_COLUMNS = ['degree_index_cv', 'degree_index_jd', 'gender_index_cv', 'job_intentstatus_cv', 'school_intersect_cvjd',
                        'location_intersect_cvjd', 'degree_match_cvjd', 'workyear_match_cvjd', 'keyU']

# 文本向量特征
ARRAY_COLUMNS = ['description_tfidf_pca', 'position_tfidf_pca', 'requirement_tfidf_pca', 'kills_tfidf_pca', 'title_category_tags_tfidf_pca']

# 连续特征
CONTINUOUS_COLUMNS = ['age', 'recruit_number', 'workYear', 'min_annual_salary', 'max_annual_salary', 
                    'work_duration_mean', 'equal_job', 'parsed_desiredSalary']


In [43]:
COLUMNS.sort()
print(COLUMNS)
print(len(COLUMNS))

['age', 'degree_index_cv', 'degree_index_jd', 'degree_match_cvjd', 'description_tfidf_pca', 'equal_job', 'gender_index_cv', 'job_intentstatus_cv', 'keyU', 'kills_tfidf_pca', 'location_intersect_cvjd', 'max_annual_salary', 'min_annual_salary', 'parsed_desiredSalary', 'position_tfidf_pca', 'recruit_number', 'requirement_tfidf_pca', 'school_intersect_cvjd', 'title_category_tags_tfidf_pca', 'workYear', 'work_duration_mean', 'workyear_match_cvjd']
22


In [44]:
a = CATEGORICAL_COLUMNS + ARRAY_COLUMNS + CONTINUOUS_COLUMNS
a.sort()
print(a)
print(len(a))

['age', 'degree_index_cv', 'degree_index_jd', 'degree_match_cvjd', 'description_tfidf_pca', 'equal_job', 'gender_index_cv', 'job_intentstatus_cv', 'keyU', 'kills_tfidf_pca', 'location_intersect_cvjd', 'max_annual_salary', 'min_annual_salary', 'parsed_desiredSalary', 'position_tfidf_pca', 'recruit_number', 'requirement_tfidf_pca', 'school_intersect_cvjd', 'title_category_tags_tfidf_pca', 'workYear', 'work_duration_mean', 'workyear_match_cvjd']
22


In [45]:
all_columns = ['cv_id', 'jd_id', 'model_label'] + COLUMNS
print(all_columns)
print(len(all_columns))

['cv_id', 'jd_id', 'model_label', 'age', 'degree_index_cv', 'degree_index_jd', 'degree_match_cvjd', 'description_tfidf_pca', 'equal_job', 'gender_index_cv', 'job_intentstatus_cv', 'keyU', 'kills_tfidf_pca', 'location_intersect_cvjd', 'max_annual_salary', 'min_annual_salary', 'parsed_desiredSalary', 'position_tfidf_pca', 'recruit_number', 'requirement_tfidf_pca', 'school_intersect_cvjd', 'title_category_tags_tfidf_pca', 'workYear', 'work_duration_mean', 'workyear_match_cvjd']
25


In [46]:
len(sample_train)

54339

In [47]:
sample_train[all_columns].to_pickle('../generalization_data/sample_train_processed_processed_54339.pkl')